In [34]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.stats
import math
import os

In [37]:
all_files = os.listdir('.')#get all files in current directory 
#all_files
all_csv_files = [file for file in all_files if file.endswith('.csv')] # find only the csv files
all_csv_files

['crowdflower_results_detailed.csv',
 'genuine_accounts_users.csv',
 'social_spambots_2_users.csv',
 'traditional_spambots_1_users.csv',
 'social_spambots_2_tweets.csv',
 'fake_followers_tweets.csv',
 'social_spambots_3_users.csv',
 'traditional_spambots_1_tweets.csv',
 'social_spambots_1_tweets.csv',
 'genuine_accounts_tweets.csv',
 'traditional_spambots_2_users.csv',
 'crowdflower_results_aggregated.csv',
 'traditional_spambots_4_users.csv',
 'crowdflower_results_contributors.csv',
 'social_spambots_3_tweets.csv',
 'traditional_spambots_3_users.csv',
 'fake_followers_users.csv',
 'social_spambots_1_users.csv']

In [39]:
list_of_dataframes = []
for file in all_csv_files:
    #print(file)
    list_of_dataframes.append(pd.read_csv(file))
list_of_dataframes

[       crowdflower_id          channel   trust  contributor_id    class  \
 0           911068393           neodev  0.8800        11057995  genuine   
 1           911068393           neodev  0.9200        11066121  genuine   
 2           911068393           neodev  0.9200        28375885  genuine   
 3           911068394        clixsense  0.9000        31001914  genuine   
 4           911068394            elite  0.8750        31410280  genuine   
 5           911068394           neodev  0.8800        35002805  genuine   
 6           911068395        clixsense  0.8400        30381282  spambot   
 7           911068395        clixsense  0.8500        32568930  spambot   
 8           911068395        clixsense  0.8000        34818655  genuine   
 9           911068396           neodev  0.9200        28375885  genuine   
 10          911068396        clixsense  0.9000        31001914  genuine   
 11          911068396           neodev  0.8000        33155316  genuine   
 12         

In [20]:
list_of_dataframes = []
for filename in all_csv_files:
    print(filename)
    

crowdflower_results_detailed.csv
Bot_Wrangler.ipynb
genuine_accounts_users.csv
social_spambots_2_users.csv
traditional_spambots_1_users.csv
social_spambots_2_tweets.csv
fake_followers_tweets.csv
social_spambots_3_users.csv
traditional_spambots_1_tweets.csv
social_spambots_1_tweets.csv
genuine_accounts_tweets.csv
traditional_spambots_2_users.csv
crowdflower_results_aggregated.csv
traditional_spambots_4_users.csv
.ipynb_checkpoints
crowdflower_results_contributors.csv
social_spambots_3_tweets.csv
traditional_spambots_3_users.csv
fake_followers_users.csv
social_spambots_1_users.csv


In [12]:
df.columns

Index(['id', 'text', 'source', 'user_id', 'truncated', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'in_reply_to_screen_name', 'retweeted_status_id',
       'geo', 'place', 'contributors', 'retweet_count', 'reply_count',
       'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive',
       'num_hashtags', 'num_urls', 'num_mentions', 'created_at', 'timestamp',
       'crawled_at', 'updated'],
      dtype='object')

In [13]:
df.head()

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,...,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated
0,593932392663912449,RT @morningJewshow: Speaking about Jews and co...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",678033.0,NaN,0.0,0.0,NaN,5.939322e+17,NaN,...,NaN,NaN,NaN,0.0,0.0,1.0,Fri May 01 00:18:11 +0000 2015,2015-05-01 02:18:11,2015-05-01 12:57:19,2015-05-01 12:57:19
1,593895316719423488,This age/face recognition thing..no reason pla...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",678033.0,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,Thu Apr 30 21:50:52 +0000 2015,2015-04-30 23:50:52,2015-05-01 12:57:19,2015-05-01 12:57:19
2,593880638069018624,Only upside of the moment I can think of is th...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",678033.0,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,2.0,0.0,0.0,Thu Apr 30 20:52:32 +0000 2015,2015-04-30 22:52:32,2015-05-01 12:57:19,2015-05-01 12:57:19
3,593847955536252928,If you're going to think about+create experien...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",678033.0,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,2.0,0.0,0.0,Thu Apr 30 18:42:40 +0000 2015,2015-04-30 20:42:40,2015-05-01 12:57:19,2015-05-01 12:57:19
4,593847687847350272,Watching a thread on FB about possible future ...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",678033.0,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,Thu Apr 30 18:41:36 +0000 2015,2015-04-30 20:41:36,2015-05-01 12:57:19,2015-05-01 12:57:19
